# Entries Module

Copyright 2023 Michael George (AKA Logiqx).

This file is part of [wsw-results](https://github.com/Logiqx/wsw-results) and is distributed under the terms of the GNU General Public License.

wsw-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

wsw-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

### Summary

Adhoc script to determine the correctness of UKWA flags in the current year

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import json
import csv

from datetime import datetime

from name import Name
from fuzzy import FuzzyMatch

import re

In [2]:
PROJDIR = os.path.realpath(os.path.join(sys.path[0], '..'))

## Process Entrants

Process all names

In [3]:
def loadConfig(year, configFile):
    '''loadConfig reads entries config (JSON) into a dictionary'''

    jsonPath = os.path.join(PROJDIR, 'events', str(year), 'config', configFile)

    with open(jsonPath, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        config = json.loads(jsonTxt)
        
    return config

In [4]:
def loadEntries(year):
    '''loadEntries reads WSW entries into a list'''

    csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', 'entries.csv')

    entrants = []
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for entrant in csvReader:
            entrants.append(entrant)

    return entrants

In [5]:
def loadEntrants(year):
    '''loadEntrants reads WSW entrants into a list'''

    csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', 'entrants.csv')

    entrants = []
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for entrant in csvReader:
            entrants.append(entrant)

    return entrants

In [6]:
def loadUkwaNames(year):
    '''loadUkwaNames reads UKWA member names into a list'''

    csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', 'ukwa.csv')

    entrants = []
    with open(csvPath, 'r', encoding='utf-8') as f:
        csvReader = csv.reader(f)
        for entrant in csvReader:
            name = Name(entrant[0])
            entrants.append(name)
            
    return entrants

In [7]:
def checkUkwaNames(entrants, memberNames, reportFuzzyMatch=False):
    '''checkUkwaNames uses fuzzy name matching to check UKWA status'''

    fuzzyMatch = FuzzyMatch()
    
    yes = []
    no = []

    fuzzy = 0
    for entrant in entrants:
        entrantName = Name(entrant['First Name'] + ' ' + entrant['Family Name'])
        
        match = None
        for memberName in memberNames:
            if fuzzyMatch.matchNameObjects(entrantName, memberName):
                match = memberName
                
                if entrantName.name != memberName.name and reportFuzzyMatch:
                    print('Fuzzy match detected {} (WSW) vs {} (UKWA)'.format(entrantName.name, memberName.name))
                    fuzzy += 1
                    
        if  entrant['UKWA Member'][:1].upper() == 'N' and match and entrant['Craft Type'] in ['Sailboard', 'Wingboard']:
            yes.append('{} ({}) - {}'.format(entrantName.name, entrant['ID'], entrant['Craft Type']))
            
        if entrant['UKWA Member'][:1].upper() == 'Y' and not (match and entrant['Craft Type'] in ['Sailboard', 'Wingboard']):
            no.append('{} ({}) - {}'.format(entrantName.name, entrant['ID'], entrant['Craft Type']))

    if fuzzy:
        print()

    if yes:
        print('People who have not marked themselves as UKWA but are a member:')
        for entrant in yes:
            print(entrant)
        print()

    if no:
        print('People who have marked themselves as UKWA but are NOT a member:')
        for entrant in no:
            print(entrant)
        print()

In [8]:
def checkFilters(entry, filters):
    '''checkFilters determines whether the filters apply to the entry'''
    
    include = True
    for filter in filters:
        if entry[filter] != filters[filter]:
            include = False
            break
                
    return include

In [9]:
def applyMappings(entry, entrant, mappings):
    '''applyMappings populates entrant with data from entry via field mappings'''
    
    for mapping in mappings:
        entrant[mapping] = entry[mappings[mapping]]

In [10]:
def applyDerivations(entry, entrant, derivations):
    '''applyDerivations populates entrant with data from entry via field derivations'''
    
    for derivation in derivations:
        for value in derivations[derivation]:
            filters = derivations[derivation][value]['Filters']
            include = checkFilters(entry, filters)
            if include:
                entrant[derivation] = value
                break

In [11]:
def applyDefaults(entrant, defaults):
    '''applyDefaults populates entrant with default values'''
    
    for default in defaults:
        entrant[default] = defaults[default]

In [12]:
def applyIdOffset(entrant, idOffset):
    '''applyIdOffset ensures that different craft use different ranges of IDs'''
    
    entrant['ID'] = int(entrant['ID']) + idOffset

In [13]:
def applyMotionId(entry, entrant, motionId):
    '''applyMotionId extracts the motion ID from the text string'''
    
    entrant['Entrant ID'] = entrant['ID']

    sep = '-'
    if sep in entry[motionId]:
        value = entry[motionId].split(sep)[-1].strip()
    else:
        value = 0

    entrant['Motion ID'] = value

    familyName = ''.join(re.findall("[a-zA-Z]+", entrant['Family Name']))
    firstName = ''.join(re.findall("[a-zA-Z]+", entrant['First Name']))

    entrant['File ID'] = '{}{}{}'.format(familyName[:3].upper(), value, firstName[:3].upper())

In [14]:
def processCraftTypes(entry, template, craftTypes, entrants):
    '''processCraftTypes creates individual entrants for each craft type'''
    
    for craftType in craftTypes:
        filters = craftType['Filters']
        include = checkFilters(entry, filters)

        if include:
            entrant = template.copy()

            defaults = craftType['Defaults']
            applyDefaults(entrant, defaults)

            mappings = craftType['Mappings']
            applyMappings(entry, entrant, mappings)

            idOffset = craftType['ID Offset']
            applyIdOffset(entrant, idOffset)

            motionId = craftType['Motion ID']
            applyMotionId(entry, entrant, motionId)
            
            entrants.append(entrant)

In [15]:
def getEntrants(entries, entriesConfig, eventConfig):
    '''getEntrants converts entries into list of entrants'''

    entrants = []
    
    startDate = eventConfig['Event']['Start Date']
    endDate = eventConfig['Event']['End Date']

    for entry in entries:
        filters = entriesConfig['Filters']
        include = checkFilters(entry, filters)
        
        if include:
            entrant = {}
            
            entrant['Start Date'] = startDate
            entrant['End Date'] = endDate

            defaults = entriesConfig['Defaults']
            applyDefaults(entrant, defaults)
            
            mappings = entriesConfig['Mappings']
            applyMappings(entry, entrant, mappings)

            derivations = entriesConfig['Derivations']
            applyDerivations(entry, entrant, derivations)
            
            craftTypes = entriesConfig['Craft Types']
            processCraftTypes(entry, entrant, craftTypes, entrants)
            
    return entrants

In [16]:
def saveOutputs(year, entrants, entriesConfig):
    '''saveOutputs generates the final CSV files'''
    
    for output in entriesConfig['Outputs']:
        file = output['File']
        csvPath = os.path.join(PROJDIR, 'events', str(year), 'config', file)

        with open(csvPath, 'w', encoding='utf-8') as outfile:
            csvWriter = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL, lineterminator=os.linesep)

            fields = output['Fields']
            csvWriter.writerow(fields)

            for entrant in entrants:
                row = []

                for field in fields:
                    if field in entrant:
                        row.append(entrant[field])
                    else:
                        row.append('')
                
                csvWriter.writerow(row)

In [17]:
def main():
    year = datetime.today().year
    
    eventConfig = loadConfig(year, 'event.json')
    entriesConfig = loadConfig(year, 'entries.json')
    entries = loadEntries(year)
    ukwaNames = loadUkwaNames(year)

    entrants = getEntrants(entries, entriesConfig, eventConfig)
    
    checkUkwaNames(entrants, ukwaNames)
    
    saveOutputs(year, entrants, entriesConfig)
    
    print('{} entrants processed'.format(len(entrants)))

if __name__ == '__main__':
    main()

People who have marked themselves as UKWA but are NOT a member:
Andrew Twinn (1194) - Sailboard
Spenser Arnold (1220) - Sailboard
Paul Arnold (1228) - Sailboard
Joe Adams (3229) - Wingboard

72 entrants processed


## All Done!